# Lab Assignment Five: Evaluation and Multi-Layer Perceptron
## Rupal Sanghavi, Omar Roa

# Preparation

## Business Case REDO THIS TO DO WITH MARKETING

This dataset represents the responses from students and their friends(ages 15-30, henceforth stated as "young people") of a Statistics class from the Faculty of Social and Economic Sciences at The Comenius University in Bratislava, Slovakia. Their survey was a mix of various topics.

* Music preferences (19 items)
* Movie preferences (12 items)
* Hobbies & interests (32 items)
* Phobias (10 items)
* Health habits (3 items)
* Personality traits, views on life, & opinions (57 items)
* Spending habits (7 items)
* Demographics (10 items)

The dataset can be found here. https://www.kaggle.com/miroslavsabo/young-people-survey

Our target is to predict how likely a "young person" would be interested in shopping at a large shopping center. We were not given details about what a "large" shopping center, but searching online for malls led us to the Avion Shopping Park in Ružinov, Slovakia. It has an area of 103,000m<sup>2</sup> and is the largest shopping mall in Slovakia (https://www.avion.sk/sk-sk/about-the-centre/fakty-a-cisla). 

Slovakia is in the lower half of European nations by size (28/48 - https://en.wikipedia.org/wiki/List_of_European_countries_by_area) and is very mountainous, making real estate space a precious commodity. This information would be of great interest to any commercial devlopment firm deciding on where to build their next shopping center or place of business. This could also help other parties trying to purchase real estate for youth-orientated construction (parks, recreation centers).

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline 
%load_ext memory_profiler
from sklearn.metrics import accuracy_score
from scipy.special import expit
import time
import math
from memory_profiler import memory_usage

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression as SKLogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer

target_classifier = 'Shopping centres'
df = pd.read_csv('responses.csv', sep=",")

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


## Define and Prepare Class Variables 

In [25]:
# remove rows whose target classfier value is NaN
df_cleaned_classifier = df[np.isfinite(df[target_classifier])]
# change NaN number values to the mean
df_imputed = df_cleaned_classifier.fillna(df.mean())
# get categorical features
object_features = list(df_cleaned_classifier.select_dtypes(include=['object']).columns)
# one hot encode categorical features
one_hot_df = pd.concat([pd.get_dummies(df_imputed[col],prefix=col) for col in object_features], axis=1)
# drop object features from imputed dataframe
df_imputed_dropped = df_imputed.drop(object_features, 1)
frames = [df_imputed_dropped, one_hot_df]
# concatenate both frames by columns
df_fixed = pd.concat(frames, axis=1)

In [26]:
df_cleaned_classifier.isnull().sum().max()

20

In [27]:
df_fixed.shape

(1008, 173)

Our dataset (1010 rows and 150 columns) was mostly ordinal data as numbers (preferences ranked 1-5) . We also had some ordinal data as strings. 

e.g.
How much time do you spend online?: No time at all - Less than an hour a day - Few hours a day - Most of the day

We first removed any rows which contained NaN values for our target classifer, Shopping centres. Afterwards we imputed mean values for any NaN values in other features. We decided to impute due to the fact that there were not many NaN values in our features compared to the size of our data set. (At most was 20 for a feature, as shown above). We then one-hot encoded any string object, which created extra features.

We are left with numerical values for our features and a size of 1008 x 173

# Evaluation

## Metrics To Evaluate Algorithm's Generalization Performance

In [30]:
# Research on Cost Matrix
# http://www.ibm.com/support/knowledgecenter/SSEPGG_11.1.0/com.ibm.im.model.doc/c_cost_matrix.html

cost_matrix = np.matrix([[0,1,2,3,4],
                        [1,0,1,2,3],
                        [3,1,0,1,2],
                        [5,3,1,0,1],
                        [7,5,2,1,0]])

def get_confusion_costTot(confusion_matrix, cost_matrix):
    score = np.sum(confusion_matrix*cost_matrix)
    return score

confusion_scorer = make_scorer(get_confusion_costTot, greater_is_better=False)

## Insert explanation of scorer here here

## Divide Data into Training and Testing

We are using Stratified Shuffle Splits so we can get a good random mix of training and testing data. We want to split and randommize our data to prevent overfitting our model. Our split uses a 20/80 split for testing and training. This gives us a significant representation of data for training and testing. Upon further research, splitting our testing data into testing and validation seems further appropriate, but we are not implementing that for our work.

Further down in our work, we are using Principal Component Analysis (via a Pipeline object) to reduce the number of features. 

In [29]:
from sklearn.model_selection import StratifiedKFold

# we want to predict the X and y data as follows:
if target_classifier in df_fixed:
    y = df_fixed[target_classifier].values # get the label we want
    del df_fixed[target_classifier] # get rid of the class label
    X = df_fixed.values # use everything else to predict!

num_folds = 10

cv_object = StratifiedKFold(n_splits= num_folds, random_state=None, shuffle=True)
cv_object.split(X,y)

print(cv_object)    

StratifiedKFold(n_splits=10, random_state=None, shuffle=True)
